<a href="https://colab.research.google.com/github/AMHlocal/final-project/blob/gs-branch/Cloud-ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**The code in the first cell will update the Spark version**




In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 http://archive.ubuntu.com/ubu

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLProjectAnalysis").getOrCreate()

**Load in connection to S3 cloud data**

In [17]:
from pyspark import SparkFiles
# Load in tsv reviews from S3 into a DataFrame
url = "https://cobrakaireviews.s3.us-east-2.amazonaws.com/amazon_reviews_us_Tools_v1_00.tsv"
spark.sparkContext.addFile(url)

# https://stackoverflow.com/questions/61799363/read-tsv-file-in-pyspark
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv"), sep=r'\t', header=True)

df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

**Remove vine reviews rows**

In [26]:
df_filtered = df[df['vine'] == "N"]
# df_filtered.show(20)
df_filtered.describe().show(20)

+-------+-----------+--------------------+--------------+--------------------+-------------------+--------------------+----------------+-----------------+------------------+------------------+-------+-----------------+--------------------+--------------------+-----------+
|summary|marketplace|         customer_id|     review_id|          product_id|     product_parent|       product_title|product_category|      star_rating|     helpful_votes|       total_votes|   vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+--------------------+--------------+--------------------+-------------------+--------------------+----------------+-----------------+------------------+------------------+-------+-----------------+--------------------+--------------------+-----------+
|  count|    1733334|             1733334|       1733334|             1733334|            1733334|             1733334|         1733334|          1733334|           1733334|        

**Transform DataFrame**

In [27]:
# product id dataframe
productid_df = df_filtered.groupby(["product_id", "product_title"]).agg({"star_rating": "avg", "helpful_votes":"count"})
productid_df.show(10)

+----------+--------------------+------------------+--------------------+
|product_id|       product_title|  avg(star_rating)|count(helpful_votes)|
+----------+--------------------+------------------+--------------------+
|B000JNNWQ2|IRWIN Tools VISE-...| 4.684863523573201|                 403|
|B000OECQOK|Titan 17005 Stain...| 4.810126582278481|                 158|
|B0045W0MD4|The American Red ...|4.5042918454935625|                 466|
|B00DQQ9ET8|de.power DP-010AA...|4.6923076923076925|                  13|
|B001P30BQE|DEWALT DG5204 Pro...|4.5646258503401365|                 294|
|B0002TKID8|Channellock GLS-1...|4.6976744186046515|                  43|
|B003O684GO|Tajima AC-500B 3/...|               5.0|                   4|
|B00T96T070|UltraFire WF-68B ...|               4.5|                   4|
|B001445114|Stiletto AG-102 A...|               4.0|                  24|
|B00002X1ZX|Stanley Tools 28-...| 4.380952380952381|                  84|
+----------+--------------------+-----

In [28]:
from pyspark.sql.functions import desc
productid_df = productid_df.withColumnRenamed("count(helpful_votes)", "total_helpful_votes")\
                                     .withColumnRenamed("avg(star_rating)", "avg_rating")
productid_df.orderBy(desc("total_helpful_votes")).show(40)

+----------+--------------------+------------------+-------------------+
|product_id|       product_title|        avg_rating|total_helpful_votes|
+----------+--------------------+------------------+-------------------+
|B006E0QAFY|UltraFire? 1000 L...| 4.268136784915308|               9387|
|B001T7QJ9O|Howard Leight Imp...| 4.445344129554656|               5434|
|B000IKK0OI|CH Hanson 0 Stud ...|4.6162816775061675|               4459|
|B000O8OTNC|Smith's PP1 Pocke...| 4.347020530796194|               3994|
|B00004VWKQ|AccuSharp Knife S...| 4.628138974888201|               2907|
|B0010O748Q|SE FS374 All-Weat...| 4.045292439372325|               2804|
|B000RB38X8|SE MZ101B Helping...| 3.880326046683957|               2699|
|B003UCODIA|SE MH1047L Illumi...| 4.151193633952254|               2639|
|B002RLR0EY|DEWALT DC970K-2 1...| 4.669767441860465|               2580|
|B0013092CS|3M WorkTunes Wire...| 4.017305315203956|               2427|
|B0018S4XIS|     Rayovac Lantern| 4.613266583229036